In [ ]:
%matplotlib inline

import os
import sys
import pathlib

import polars as pl
import pandas as pd
import numpy as np
import matplotlib as plt
import mplfinance as mpf
from tqdm import tqdm
from datetime import date, timedelta, datetime, timezone
import json

from scipy.optimize import curve_fit
import optuna

from plot_utils import plot_correlation
from constants import target_symbols

In [ ]:
def load_timebar(symbol: str="BTCUSDT", last_date: date=(datetime.now(timezone.utc) - timedelta(days = 3)).date(), days: int=365, interval_sec: int=14400) -> pl.DataFrame:
    """
    タイムバーを読み込み、データフレームを返す関数。あらかじめdownload_trades.pyとgenerate_timebar.pyを使ってデータを準備しておくこと

    Args:
        symbol (str): 通貨ペア
        last_date (date): 読み込む最後の日付
        days (int): 読み込む日数
        interval_sec (int): インターバル（秒）
    Returns:
        pl.DataFrame: タイムバーのデータフレーム
    """
    _start_date = last_date - timedelta(days=days)
    
    _df_timebar = None
    _date_cursor = _start_date

    while _date_cursor <= last_date:
        if not pathlib.Path(f"/home/jovyan/work/binance_utility/timebar/{symbol}_TIMEBAR_{interval_sec:d}SEC_{_date_cursor.year:04d}-{_date_cursor.month:02d}-{_date_cursor.day:02d}.parquet").exists():
            _date_cursor = _date_cursor + timedelta(days=1)
            continue
        
        if _df_timebar is None:
            _df_timebar = pl.read_parquet(f"/home/jovyan/work/binance_utility/timebar/{symbol}_TIMEBAR_{interval_sec:d}SEC_{_date_cursor.year:04d}-{_date_cursor.month:02d}-{_date_cursor.day:02d}.parquet")
        else:
            _df_timebar = _df_timebar.vstack(pl.read_parquet(f"/home/jovyan/work/binance_utility/timebar/{symbol}_TIMEBAR_{interval_sec:d}SEC_{_date_cursor.year:04d}-{_date_cursor.month:02d}-{_date_cursor.day:02d}.parquet"))
        _date_cursor = _date_cursor + timedelta(days=1)

    _df_timebar = _df_timebar.with_columns([(pl.col("market_bid_volume") - pl.col("market_ask_volume")).alias("signed_volume"),
                                            ((pl.col("high") - pl.col("close")) / pl.col("high")).alias("high_close_pct"),
                                            ((pl.col("close") - pl.col("low")) / pl.col("low")).alias("low_close_pct"),
                                            pl.col("close").ewm_std(30).alias("ewma_std")]).drop_nulls()
    return _df_timebar

In [ ]:
def calc_param(df_timebar: pl.DataFrame, symbol: str="BTCUSDT", interval_sec: int=14400) -> tuple[float, float, float, float]:
    _x = df_timebar["signed_volume"]
    _y = df_timebar["delta_price"]

    def _func(x, a, b):
        return np.where(np.abs(x) < 1, 0, np.sign(x) * a * np.log(b * np.abs(x)))
    
    _popt, _pcov = curve_fit(_func, _x, _y)
    _param_b = _popt[0]
    _param_c = _popt[1]
    
    _array_histogram = np.histogram(df_timebar["signed_volume"], bins=256, density=True)
    _density_quantile = 0.97
    _density_upperlimit = np.quantile(_array_histogram[0], _density_quantile)
    _df_histogram = pl.DataFrame({"signed_volume": _array_histogram[1][:-1], "density": _array_histogram[0]}).filter(pl.col("density") > 0).filter(pl.col("density") < _density_upperlimit)

    def _func2(x, a, b):
        return a * np.power(np.abs(x), -1-b)
    
    _popt, _pcov = curve_fit(_func2, _df_histogram["signed_volume"], _df_histogram["density"])

    _param_a = _popt[0]
    _param_alpha = _popt[1]

    return _param_a, _param_b, _param_c, _param_alpha

In [ ]:
# bid / askヒット時間を計算するユーティリティ関数
# timestamp_toは含まない

_cache_date = None
_cache_df_trades = None

def calc_hit_time(timestamp_from, timestamp_to, bid_price, ask_price, symbol):
    global _cache_date, _cache_df_trades
    _bid_hit_time = np.nan
    _ask_hit_time = np.nan

    if np.isnan(bid_price) == True and np.isnan(ask_price) == True:
        return _bid_hit_time, _ask_hit_time

    _date = datetime.fromtimestamp(timestamp_from // 1000, timezone.utc).date()
    if _cache_date is not None and _cache_date == _date:
        _df_trades = _cache_df_trades
    else:
        _cache_df_trades = pl.read_parquet(f"/home/jovyan/work/binance_utility/trades/{symbol}_TRADES_{_date.year:04d}-{_date.month:02d}-{_date.day:02d}.parquet")
        _cache_date = _date
        _df_trades = _cache_df_trades
        
    _df_trades_filtered = _df_trades.filter((pl.col("timestamp") >= timestamp_from) & (pl.col("timestamp") < timestamp_to))
    _df_trades_less_than_bid_price = _df_trades_filtered.filter(pl.col("price") <= bid_price)
    if _df_trades_less_than_bid_price.shape[0] >= 1:
        _bid_hit_time = _df_trades_less_than_bid_price["timestamp"].min()
    
    _df_trades_more_than_ask_price = _df_trades_filtered.filter(pl.col("price") >= ask_price)
    if _df_trades_more_than_ask_price.shape[0] >= 1:
        _ask_hit_time = _df_trades_more_than_ask_price["timestamp"].min()
    
    return _bid_hit_time, _ask_hit_time

# Buyを実行するユーティリティ関数
def buy(q, cash, delta, position_abs_max, price, fee_rate):
    _q, _cash = q, cash
    if _q < position_abs_max:
        _delta = delta if q + delta <= position_abs_max else position_abs_max - _q
        #if _cash >= _delta * price:
        _q = q + _delta
        _cash = cash - (1 + fee_rate) * _delta * price
    return _q, _cash

# Sellを実行するユーティリティ関数
def sell(q, cash, delta, position_abs_max, price, fee_rate):
    _q, _cash = q, cash
    if _q > -position_abs_max:
        _delta = delta if q - delta >= -position_abs_max else np.abs(-position_abs_max - _q)
        _q = q - _delta
        _cash = cash + (1 - fee_rate) * _delta * price
    
    return _q, _cash

In [ ]:
def simulation(df_timebar: pl.DataFrame, param_a: float, param_alpha: float, param_b: float, param_c: float, gamma: float, maker_fee_rate: float=0.02/100, taker_fee_rate: float=0.04/100, symbol: str="BTCUSDT", interval_sec: int=14400):
    _initial_cash = df_timebar[0, "open"]

    # 指値幅の計算パラメータ
    params = {
        "position_abs_max": 1,
        "lambda": 1 / interval_sec,
        "delta": 1,
        "a": param_a,
        "alpha": param_alpha,
        "b": param_b,
        "c": param_c,
        "gamma": gamma
    }

    params["A"] = params["lambda"] / params["alpha"] * params["a"] / params["c"]
    params["k"] = params["alpha"] / params["b"]
    params["1/k"] = 1 / params["k"]

    #print(json.dumps(params, indent=4, sort_keys=True))

    _df_timebar = df_timebar.with_columns([
        pl.lit(np.nan).alias("bid_price"),
        pl.lit(np.nan).alias("ask_price"),
        pl.lit(0.0).alias("trade_count"),
        pl.lit(0.0).alias("q"),
        pl.lit(0.0).alias("cash"),
        pl.lit(0.0).alias("value")])
    _df_timebar[0, "cash"] = _initial_cash

    # 指値幅の計算 (_qは_idxのものを使い、指値は_idx+1に入れる)
    _df_timebar_num_rows = _df_timebar.shape[0]
    _pbar = tqdm(range(0, _df_timebar_num_rows))
    for _idx in _pbar:
    #for _idx in range(0, _df_timebar_num_rows):
        # このバーが始まった時点でのポジションとキャッシュを取得
        _q = _df_timebar[_idx, "q"]
        _cash = _df_timebar[_idx, "cash"]
        _sigma = _df_timebar[_idx, "weighted_std"]

        # 最後の行は指値幅計算をしないweighted_stdeighted_std
        if _idx < _df_timebar_num_rows - 1:
            _bid_delta = 1 / params["k"] + (2 * _q + params["delta"]) / 2 * np.sqrt((params["gamma"] * _sigma ** 2 * np.e) / (2 * params["A"] * params["delta"] * params["k"]))
            _ask_delta = 1 / params["k"] - (2 * _q - params["delta"]) / 2 * np.sqrt((params["gamma"] * _sigma ** 2 * np.e) / (2 * params["A"] * params["delta"] * params["k"]))
            #_gamma_delta = params["gamma"] * params["delta"]
            #_bid_delta = 1 / _gamma_delta * np.log(1 + _gamma_delta / params["k"]) + (2 * _q + params["delta"]) / 2 * np.sqrt((params["gamma"] * _sigma ** 2) / (2 * params["A"] * params["delta"] * params["k"]) * (1 + _gamma_delta / params["k"]) ** (params["k"] / _gamma_delta + 1))
            #_ask_delta = 1 / _gamma_delta * np.log(1 + _gamma_delta / params["k"]) - (2 * _q - params["delta"]) / 2 * np.sqrt((params["gamma"] * _sigma ** 2) / (2 * params["A"] * params["delta"] * params["k"]) * (1 + _gamma_delta / params["k"]) ** (params["k"] / _gamma_delta + 1))
            _df_timebar[_idx + 1, "bid_price"] = _df_timebar[_idx, "close"] - _bid_delta
            _df_timebar[_idx + 1, "ask_price"] = _df_timebar[_idx, "close"] + _ask_delta            

        # 最初の行は指値幅がないのでトレードの計算をしない                                
        if _idx == 0:
            _df_timebar[_idx+1, "q"] = _q
            _df_timebar[_idx+1, "cash"] = _cash
            _df_timebar[_idx+1, "value"] = _cash + _q * _df_timebar[_idx, "close"]
            continue
        
        # _idx-1で計算され、_idxに入っている指値をもとにヒットを計算する
        _is_bid_hit = _df_timebar[_idx, "bid_price"] >= _df_timebar[_idx, "low"]
        _is_ask_hit = _df_timebar[_idx, "ask_price"] <= _df_timebar[_idx, "high"]

        if _is_bid_hit and _is_ask_hit:
            #print("maker buy and maker sell")
            _current_timestamp = _df_timebar[_idx, "timestamp"]
            _bid_hit_timestamp, _ask_hit_timestamp = calc_hit_time(_current_timestamp, _current_timestamp + interval_sec * 1000, _df_timebar[_idx, "bid_price"], _df_timebar[_idx, "ask_price"], _symbol)
            if _bid_hit_timestamp <= _ask_hit_timestamp:
                _q, _cash = buy(_q, _cash, params["delta"], params["position_abs_max"], _df_timebar[_idx, "bid_price"], maker_fee_rate)
                _q, _cash = sell(_q, _cash, params["delta"], params["position_abs_max"], _df_timebar[_idx, "ask_price"], maker_fee_rate)
                _df_timebar[_idx, "trade_count"] = 2
            else:
                _q, _cash = sell(_q, _cash, params["delta"], params["position_abs_max"], _df_timebar[_idx, "ask_price"], maker_fee_rate)
                _q, _cash = buy(_q, _cash, params["delta"], params["position_abs_max"], _df_timebar[_idx, "bid_price"], maker_fee_rate)
                _df_timebar[_idx, "trade_count"] = 2
        elif _is_bid_hit:
            #print("maker buy and taker sell")
            _q, _cash = buy(_q, _cash, params["delta"], params["position_abs_max"], _df_timebar[_idx, "bid_price"], maker_fee_rate)
            _q, _cash = sell(_q, _cash, params["delta"], params["position_abs_max"], _df_timebar[_idx, "close"], taker_fee_rate)
            _df_timebar[_idx, "trade_count"] = 2
        elif _is_ask_hit:
            #print("maker sell and taker buy")
            _q, _cash = sell(_q, _cash, params["delta"], params["position_abs_max"], _df_timebar[_idx, "ask_price"], maker_fee_rate)
            _q, _cash = buy(_q, _cash, params["delta"], params["position_abs_max"], _df_timebar[_idx, "close"], taker_fee_rate)
            _df_timebar[_idx, "trade_count"] = 2
        
        # Valueの計算などを行い、現タイムバー完了時点のタイムバーのq, cash, valueを_idx+1に代入しておく
        # 最後の行はこの計算をしない
        if _idx < _df_timebar_num_rows - 1:
            _df_timebar[_idx+1, "q"] = _q
            _df_timebar[_idx+1, "cash"] = _cash
            _df_timebar[_idx+1, "value"] = _cash + _q * _df_timebar[_idx, "close"]
            _pbar.set_description(f"value={_df_timebar[_idx+1, 'value']:07.6f}")
    
    return _df_timebar

def objective_param(df_timebar, param_a, param_alpha, param_b, param_c, symbol):
    def objective(trial):
        _gamma = trial.suggest_float(low=0.0, high=1.0, name="gamma")
        _df_timebar = simulation(df_timebar=df_timebar, param_a=param_a, param_alpha=param_alpha, param_b=param_b, param_c=param_c, gamma=_gamma, symbol=symbol)
        
        # 表示用データフレームの作成
        if False:
            _df_timebar_pd = _df_timebar.to_pandas().dropna()
            _df_timebar_pd["datetime"] = pd.to_datetime(df_timebar_pd["timestamp"], unit="ms")
            
            _df_timebar_pd = _df_timebar_pd.set_index("datetime", drop = True)
            _df_timebar_pd_filtered = _df_timebar_pd[(_df_timebar_pd.index >= "2022-5-1") & (_df_timebar_pd.index <= "2023-6-1")]

            # プロット
            additional_plot = [mpf.make_addplot(
                _df_timebar_pd_filtered["value"], type="line", color="green", ylabel="Value [USDT]", width=1, panel=2),
                #mpf.make_addplot(df_timebar_pd_filtered["q"], type="line", color="green", ylabel="position", width=1, panel=3),
                mpf.make_addplot(_df_timebar_pd_filtered["bid_price"], type="line", color="blue", width=0.2),
                mpf.make_addplot(_df_timebar_pd_filtered["ask_price"], type="line", color="red", width=0.2),
                mpf.make_addplot(_df_timebar_pd_filtered["weighted_std"], type="line", color="red", ylabel="weighted_std", width=0.2, panel = 3)]
            mpf.plot(_df_timebar_pd_filtered, type="candle", title=f"{symbol} MM strategy gamma={_gamma:.04f}", ylabel="Price [USDT]", style="yahoo", figsize=(8, 4), addplot = additional_plot, volume=True, warn_too_much_data=int(365*24*24/2))
        
        _df_timebar = _df_timebar.with_columns((np.log(pl.col("value")) - np.log(pl.col("value").shift(1))).alias("logreturn"))
        _df_timebar = _df_timebar.with_columns(pl.when(pl.col("logreturn").is_infinite()).then(None).otherwise(pl.col("logreturn")).keep_name()).drop_nulls()
        _return = _df_timebar["logreturn"].mean()
        _risk = _df_timebar["logreturn"].std()
        _sr = _return / _risk if np.isnan(_risk) == False and np.isnan(_return) == False and _risk != 0 else 0

        print(f"gamma={_gamma}, trade_count={_df_timebar['trade_count'].sum()}, final_value = {_df_timebar[-1, 'value']}, Return = {_return:.04f}, Risk={_risk:.04f}, SR = {_sr:.04f}")

        return _sr
    return objective

In [8]:
dic_timebars = {}

for _symbol in target_symbols:
    dic_timebars[_symbol] = load_timebar(symbol = _symbol)
    _param_a, _param_b, _param_c, _param_alpha = calc_param(dic_timebars[_symbol], symbol = _symbol)
    
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_param(dic_timebars[_symbol], _param_a, _param_alpha, _param_b, _param_c, _symbol), n_trials=100)
    print(f"best params = {study.best_params}, best value = {study.best_value}")
    #simulation(dic_timebars[_symbol], _param_a, _param_alpha, _param_b, _param_c, 0.0002, symbol=_symbol)


dic_timebars

value=-280.159115:  97%|█████████▋| 2123/2196 [00:10<00:00, 177.30it/s]